In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('itamplify_train.csv')
# drop X and track_id columns
columns_to_drop = ['track_name', 'track_id', 'artists', 'album_name', 'X']
data = data.drop(columns=columns_to_drop, axis=1)
data.head()

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,64,168000,0,0.443,0.778,0,-7.564,0,0.2660,0.241,0.000000,0.2150,0.628,128.250,5,afrobeat
1,44,318360,0,0.580,0.800,6,-7.528,1,0.3450,0.281,0.000000,0.0708,0.483,81.303,4,afrobeat
2,42,171989,0,0.805,0.746,1,-5.211,0,0.1850,0.238,0.000000,0.2150,0.717,127.922,4,afrobeat
3,46,205440,0,0.582,0.660,7,-4.988,0,0.0544,0.399,0.000000,0.1270,0.484,83.789,4,afrobeat
4,44,220800,0,0.776,0.314,2,-9.513,1,0.0340,0.881,0.000723,0.1390,0.476,124.055,4,afrobeat


In [11]:

train, test = train_test_split(data, 
                                stratify=data['track_genre'],
                                test_size=0.2,
                                random_state=42)

train, val = train_test_split(train,
                                stratify=train['track_genre'],
                                test_size=0.2,
                                random_state=42)


train_data = TabularDataset(train)
val_data = TabularDataset(val)
test_data = TabularDataset(test)

label = 'track_genre'
predictor = TabularPredictor(label=label, path='val',
                             problem_type='multiclass',
                             eval_metric='log_loss',
                             sample_weight='balance_weight',                             
                             ).fit(
                                 use_bag_holdout=True,
                                 train_data = train_data,
                                 tuning_data=val_data,
                                 time_limit=None,
                                 presets='best_quality',
                                 num_cpus=16,
                                 num_gpus=1,
                                 hyperparameters = {
                                'NN_TORCH': {},
                                'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}, {}, 'GBMLarge'],
                                'CAT': {},
                                'XGB': {},
                                'FASTAI': {},
                                'RF': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}},
                                       {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}},
                                       {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
                                 'XT': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}},
                                        {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, 
                                        {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
                                'KNN': [{'weights': 'uniform', 'ag_args': {'name_suffix': 'Unif'}},
                                        {'weights': 'distance', 'ag_args': {'name_suffix': 'Dist'}}],
                                 }
                             )

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Using predefined sample weighting strategy: balance_weight. Evaluation metrics will ignore sample weights, specify weight_evaluation=True to instead report weighted metrics.
Beginning AutoGluon training ...
AutoGluon will save models to "val\"
AutoGluon Version:  0.8.2
Python Version:     3.10.14
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621


Disk Space Avail:   255.00 GB / 2000.38 GB (12.7%)
Train Data Rows:    5289
Train Data Columns: 15
Tuning Data Rows:    1323
Tuning Data Columns: 15
Label Column: track_genre
Preprocessing data ...
Assigning sample weights to balance differences in frequency of classes.
Train Data Class Count: 15
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    4537.25 MB
	Train Data (Original)  Memory Usage: 0.79 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 2 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fittin

In [12]:
predictor.feature_importance(val_data)

Computing feature importance via permutation shuffling for 15 features using 1323 rows with 5 shuffle sets...
	606.49s	= Expected runtime (121.3s per shuffle set)
	157.82s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
popularity,1.059372,0.036244,1.641594e-07,5,1.133999,0.984744
danceability,0.427856,0.032729,4.077222e-06,5,0.495246,0.360465
acousticness,0.356832,0.023861,2.385032e-06,5,0.405962,0.307702
instrumentalness,0.243670,0.008918,2.148795e-07,5,0.262031,0.225308
loudness,0.160003,0.004036,4.852864e-08,5,0.168313,0.151693
valence,0.155847,0.016501,1.485788e-05,5,0.189823,0.121872
speechiness,0.135176,0.006091,4.934888e-07,5,0.147718,0.122634
energy,0.102965,0.008072,4.495758e-06,5,0.119585,0.086345
duration_ms,0.095698,0.012942,3.918080e-05,5,0.122346,0.069050
tempo,0.067515,0.006956,1.333238e-05,5,0.081838,0.053193


In [13]:
predictor.feature_importance(test_data)

Computing feature importance via permutation shuffling for 15 features using 1654 rows with 5 shuffle sets...
	621.07s	= Expected runtime (124.21s per shuffle set)
	188.29s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
popularity,1.077708,0.040715,2.439834e-07,5,1.161540,0.993875
danceability,0.451207,0.007851,1.099626e-08,5,0.467373,0.435041
acousticness,0.343902,0.020281,1.444817e-06,5,0.385661,0.302142
instrumentalness,0.265088,0.015245,1.306657e-06,5,0.296477,0.233700
loudness,0.158397,0.005160,1.349416e-07,5,0.169022,0.147773
valence,0.144231,0.011976,5.653057e-06,5,0.168891,0.119571
speechiness,0.128649,0.007660,1.501263e-06,5,0.144422,0.112877
energy,0.122750,0.005922,6.481327e-07,5,0.134944,0.110556
duration_ms,0.094696,0.014213,5.910745e-05,5,0.123960,0.065431
tempo,0.061289,0.007968,3.351547e-05,5,0.077695,0.044884


In [17]:
predictor.leaderboard(val_data, extra_metrics=['mcc', 'balanced_accuracy'])

                      model  score_test       mcc  balanced_accuracy  score_val  pred_time_test  pred_time_val     fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       WeightedEnsemble_L3   -1.158661  0.608303           0.599048  -1.161157        7.765139       6.763353  2097.910263                 0.005508                0.002001           1.460847            3       True         26
1       WeightedEnsemble_L2   -1.170300  0.598834           0.591724  -1.170300        1.753766       0.722270   199.861412                 0.004506                0.002639           1.897038            2       True         14
2           CatBoost_BAG_L2   -1.191192  0.593906           0.606335  -1.191192        4.480467       3.258035  1892.913781                 0.129790                0.200474        1644.923362            2       True         20
3         LightGBMXT_BAG_L2   -1.192547  0.587186           0.588053  -1.192547        4.599

,model,score_test,mcc,balanced_accuracy,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-1.158661,0.608303,0.599048,-1.161157,7.765139,6.763353,2097.910263,0.005508,0.002001,1.460847,3,True,26
1,WeightedEnsemble_L2,-1.170300,0.598834,0.591724,-1.170300,1.753766,0.722270,199.861412,0.004506,0.002639,1.897038,2,True,14
2,CatBoost_BAG_L2,-1.191192,0.593906,0.606335,-1.191192,4.480467,3.258035,1892.913781,0.129790,0.200474,1644.923362,2,True,20
3,LightGBMXT_BAG_L2,-1.192547,0.587186,0.588053,-1.192547,4.599202,3.266953,342.635816,0.248526,0.209392,94.645398,2,True,16
4,NeuralNetFastAI_BAG_L2,-1.198348,0.606248,0.592584,-1.198348,4.681720,3.286853,261.958049,0.331043,0.229292,13.967630,2,True,15
5,XGBoost_BAG_L2,-1.199255,0.589227,0.585920,-1.199255,5.487127,3.386176,411.767775,1.136450,0.328615,163.777356,2,True,23
6,CatBoost_BAG_L1,-1.209487,0.584538,0.596907,-1.209487,0.104055,0.106128,123.214709,0.104055,0.106128,123.214709,1,True,8
7,LightGBM_BAG_L2,-1.221981,0.598483,0.593632,-1.221981,4.531412,3.247786,400.055429,0.180735,0.190225,152.065011,2,True,17
8,RandomForestEntr_BAG_L2,-1.223840,0.594300,0.575952,-1.223840,4.566032,4.027256,253.874321,0.215356,0.969695,5.883902,2,True,19
9,LightGBMXT_BAG_L1,-1.226805,0.581751,0.589977,-1.226805,0.485262,0.457482,8.615038,0.485262,0.457482,8.615038,1,True,4


In [18]:
predictor.leaderboard(test_data, extra_metrics=['mcc', 'balanced_accuracy'])

                      model  score_test       mcc  balanced_accuracy  score_val  pred_time_test  pred_time_val     fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       WeightedEnsemble_L3   -1.195932  0.577820           0.580128  -1.161157        7.880457       6.763353  2097.910263                 0.008516                0.002001           1.460847            3       True         26
1         LightGBMXT_BAG_L2   -1.208024  0.571694           0.586346  -1.192547        4.651929       3.266953   342.635816                 0.245943                0.209392          94.645398            2       True         16
2            XGBoost_BAG_L2   -1.215037  0.571388           0.583461  -1.199255        5.509368       3.386176   411.767775                 1.103381                0.328615         163.777356            2       True         23
3           CatBoost_BAG_L2   -1.219152  0.566628           0.590311  -1.191192        4.538

,model,score_test,mcc,balanced_accuracy,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-1.195932,0.577820,0.580128,-1.161157,7.880457,6.763353,2097.910263,0.008516,0.002001,1.460847,3,True,26
1,LightGBMXT_BAG_L2,-1.208024,0.571694,0.586346,-1.192547,4.651929,3.266953,342.635816,0.245943,0.209392,94.645398,2,True,16
2,XGBoost_BAG_L2,-1.215037,0.571388,0.583461,-1.199255,5.509368,3.386176,411.767775,1.103381,0.328615,163.777356,2,True,23
3,CatBoost_BAG_L2,-1.219152,0.566628,0.590311,-1.191192,4.538676,3.258035,1892.913781,0.132689,0.200474,1644.923362,2,True,20
4,WeightedEnsemble_L2,-1.220706,0.563523,0.567020,-1.170300,1.742135,0.722270,199.861412,0.005526,0.002639,1.897038,2,True,14
5,LightGBM_BAG_L2,-1.247355,0.577371,0.584554,-1.221981,4.601892,3.247786,400.055429,0.195906,0.190225,152.065011,2,True,17
6,CatBoost_BAG_L1,-1.247946,0.551938,0.568111,-1.209487,0.092888,0.106128,123.214709,0.092888,0.106128,123.214709,1,True,8
7,RandomForestEntr_BAG_L2,-1.248833,0.568985,0.566029,-1.223840,4.639825,4.027256,253.874321,0.233838,0.969695,5.883902,2,True,19
8,ExtraTreesGini_BAG_L2,-1.249023,0.573613,0.573380,-1.241051,4.670959,3.853422,249.094799,0.264972,0.795861,1.104381,2,True,21
9,LightGBMXT_BAG_L1,-1.255718,0.543033,0.560416,-1.226805,0.512393,0.457482,8.615038,0.512393,0.457482,8.615038,1,True,4


In [3]:
y_pred = predictor.predict(test_data)
y_pred_proba = predictor.predict_proba(test_data)


In [4]:
predictor.evaluate(test_data)
predictor.leaderboard(test_data)

Evaluation: accuracy on test data: 0.5973397823458283
Evaluations on test data:
{
    "accuracy": 0.5973397823458283,
    "balanced_accuracy": 0.5468865874751114,
    "mcc": 0.5622651653308025
}


                  model  score_test  score_val  pred_time_test  pred_time_val   fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2    0.597340   0.637051        1.437514       0.399318  60.762756                 0.017547                0.001000           0.670227            2       True         14
1              CatBoost    0.585248   0.614367        0.040510       0.005007  19.215830                 0.040510                0.005007          19.215830            1       True          8
2            LightGBMXT    0.581620   0.616257        0.190249       0.030145  10.315883                 0.190249                0.030145          10.315883            1       True          4
3              LightGBM    0.580411   0.606805        0.032661       0.006006   4.815328                 0.032661                0.006006           4.815328            1       True          5
4      RandomForestGini    0.579807   0.

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.597340,0.637051,1.437514,0.399318,60.762756,0.017547,0.001000,0.670227,2,True,14
1,CatBoost,0.585248,0.614367,0.040510,0.005007,19.215830,0.040510,0.005007,19.215830,1,True,8
2,LightGBMXT,0.581620,0.616257,0.190249,0.030145,10.315883,0.190249,0.030145,10.315883,1,True,4
3,LightGBM,0.580411,0.606805,0.032661,0.006006,4.815328,0.032661,0.006006,4.815328,1,True,5
4,RandomForestGini,0.579807,0.608696,0.286783,0.095191,1.117476,0.286783,0.095191,1.117476,1,True,6
5,RandomForestEntr,0.579202,0.593573,0.262219,0.109226,1.380582,0.262219,0.109226,1.380582,1,True,7
6,LightGBMLarge,0.573761,0.597353,0.515204,0.091915,16.797411,0.515204,0.091915,16.797411,1,True,13
7,XGBoost,0.566505,0.593573,0.215707,0.015063,6.316957,0.215707,0.015063,6.316957,1,True,11
8,NeuralNetFastAI,0.558646,0.567108,0.031178,0.027468,7.743517,0.031178,0.027468,7.743517,1,True,3
9,ExtraTreesEntr,0.550181,0.567108,0.401197,0.092361,1.004998,0.401197,0.092361,1.004998,1,True,10


In [5]:
predictor.feature_importance(test_data)

Computing feature importance via permutation shuffling for 15 features using 1654 rows with 5 shuffle sets...
	116.5s	= Expected runtime (23.3s per shuffle set)
	40.42s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
popularity,0.220193,0.006670,1.008954e-07,5,0.233927,0.206460
danceability,0.106167,0.005551,8.936283e-07,5,0.117597,0.094737
acousticness,0.089359,0.009022,1.230392e-05,5,0.107936,0.070782
instrumentalness,0.075212,0.006464,6.482274e-06,5,0.088521,0.061903
loudness,0.046796,0.010719,3.084047e-04,5,0.068865,0.024726
valence,0.046070,0.003095,2.428598e-06,5,0.052442,0.039698
speechiness,0.041112,0.005458,3.641858e-05,5,0.052351,0.029874
duration_ms,0.035429,0.007300,2.046027e-04,5,0.050461,0.020398
energy,0.033857,0.004545,3.803336e-05,5,0.043215,0.024500
tempo,0.025877,0.007069,6.066261e-04,5,0.040432,0.011322
